In [5]:
run src/midi_processing.py

In [6]:
run src/model.py

In [7]:
mod = load_model('data/model_60.h5')

In [8]:
uniq = np.unique(X)

In [9]:
gen = generate_from_random(uniq)

In [10]:
for i in range(600, 610):
    lst = generate_notes(gen, mod, length=i)
    try:       
        s = one_hot_to_midi(lst)
        print(f"{i} worked!")
        for note in s[0].notes:
            note.octave = 5
        mf = midi.translate.streamToMidiFile(s)
        mf.open('data/song.mid', 'wb')
        mf.write()
        mf.close()
        break
    except:
        print(f"{i} didn't work")

600 didn't work
601 didn't work
602 didn't work
603 worked!


In [14]:
for note in s[0].notes:
    note.octave = 5

In [15]:
mf = midi.translate.streamToMidiFile(s)
mf.open('data/song60.mid', 'wb')
mf.write()
mf.close()

In [16]:
s.show('text')

{0.0} <music21.stream.Part 0x7fdda03c1240>
    {0.0} <music21.instrument.Instrument 'Saxophone'>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note E->
    {0.84} <music21.note.Note C>
    {0.94} <music21.note.Note E->
    {3.42} <music21.note.Note D>
    {6.06} <music21.note.Note C>
    {6.16} <music21.note.Note E>
    {10.26} <music21.note.Note E>
    {10.78} <music21.note.Note B>
    {14.32} <music21.note.Note E->
    {15.48} <music21.note.Note A>
    {15.6} <music21.note.Note C>
    {15.7} <music21.note.Note G#>
    {16.86} <music21.note.Note G#>
    {16.88} <music21.note.Note F#>
    {17.12} <music21.note.Note E>
    {17.22} <music21.note.Note E->
    {19.56} <music21.note.Note E->
    {19.66} <music21.note.Note C>
    {19.76} <music21.note.Note D>
    {19.86} <music21.note.Note B>
    {22.5} <music21.note.Note B>
    {24.96} <music21.note.Note B>
    {27.42} <music21.note.Note D>
    {31.46} <music2